# Realtime APIを使用したコンテキスト要約
## 1. 概要
マイクの音声を聞き取り、リアルタイムで応答し、**長い会話を要約**して品質を維持する、エンドツーエンドの**音声ボット**を構築します。

### 学習内容
1. **ライブマイクストリーミング** → OpenAI *Realtime*（音声対音声）エンドポイント
2. 各ターンでの**即座の転写と音声再生**
3. **すべての**ユーザー/アシスタントメッセージを保存する**会話状態コンテナ**
4. **自動「コンテキストトリム」** – トークンウィンドウが非常に大きくなった場合（設定可能）、古いターンを要約に圧縮
5. カスタマーサポートボット、キオスク、多言語アシスタントをサポートするように適応可能な**拡張可能な設計**

### 前提条件

| 要件 | 詳細 |
|-------------|---------|
| **Python ≥ 3.10** | 問題が発生しないことを保証します |
| **OpenAI APIキー** | シェルで`OPENAI_API_KEY`を設定するか、インラインで貼り付け（*本番環境には理想的ではない*） |
| マイク + スピーカー | プロンプトが表示されたらOS権限を許可 |

**キーの設定でサポートが必要ですか？**  
> [公式クイックスタートガイド](https://platform.openai.com/docs/quickstart#step-2-set-your-api-key)に従ってください。

*注意事項:*
> 1. gpt-realtimeは32kトークンのコンテキストウィンドウをサポートしていますが、特定のユースケースでは、コンテキストウィンドウにより多くのトークンを詰め込むとパフォーマンスが低下することがあります。
> 2. トークンウィンドウ = モデルがセッション中にメモリに保持するすべてのトークン（単語と音声トークン）

### ワンライナーインストール（新しいセルで実行）

*新しいAPIパラメータ:*
> 1. Realtime API GAでは新しいパラメータ[`truncation`](https://platform.openai.com/docs/api-reference/realtime-client-events/session/update#realtime-client-events/session/update-session-realtime-session-configuration-truncation)がリリースされました。このパラメータは、関連情報を保持しながらキャッシュヒット率を最大化し、コンテキストの切り詰めを自動的に最適化します。

In [ ]:
# Run once to install or upgrade dependencies (comment out if already installed)
# !pip install --upgrade openai websockets sounddevice simpleaudio

In [1]:
# Standard library imports
import os
import sys
import io
import json
import base64
import pathlib
import wave
from dataclasses import dataclass, field
from typing import List, Literal

# Third-party imports
import asyncio
import numpy as np
import sounddevice as sd         # microphone capture
import simpleaudio               # speaker playback
import websockets                # WebSocket client
import openai                    # OpenAI Python SDK >= 1.14.0

In [2]:
# Set your API key safely
openai.api_key = os.getenv("OPENAI_API_KEY", "")
if not openai.api_key:
    raise ValueError("OPENAI_API_KEY not found – please set env var or edit this cell.")

## 2. トークン使用量 – テキスト vs 音声

大きなトークンウィンドウは貴重であり、使用する追加トークンはすべてレイテンシとコストを増加させます。  
**音声**の場合、振幅、タイミング、その他の音響的詳細を表現する必要があるため、入力トークンウィンドウはプレーンテキストよりもはるかに速く増加します。

実際には、同じ文章でも音声の場合、テキストと比較して**約10倍**のトークンが必要になることがよくあります。


* gpt-realtimeは最大**32kトークン**まで受け入れ、トークンサイズが増加すると指示への準拠が低下する可能性があります。
* ユーザー/アシスタントのターンごとにトークンを消費するため、ウィンドウは**増加し続けます**。
* **戦略**: 古いターンを単一のアシスタントメッセージに要約し、最後の数ターンをそのまま保持して続行します。

<img src="../images/text-vs-audio-tokens.png" alt="drawing" width="800"/>

## 3. ヘルパー関数
以下のヘルパー関数により、完全なスクリプトを実行できるようになります。

### 3.1 会話状態
HTTPベースのChat Completionsとは異なり、Realtime APIは2つの主要コンポーネントを持つオープンな**ステートフル**セッションを維持します：

| コンポーネント | 目的 |
|----------------|---------|
| **Session**     | グローバル設定を制御 — モデル、音声、モダリティ、VAD等 |
| **Conversation** | ユーザーとアシスタント間のターンごとのメッセージを保存 — 音声とテキストの両方 |

このノートブックでは、これらのコンポーネントを単純な`ConversationState`オブジェクト内にラップして、ロジックを整理し、履歴を追跡し、コンテキストウィンドウが満杯になった際の要約を管理します。

In [3]:
@dataclass
class Turn:
    """One utterance in the dialogue (user **or** assistant)."""
    role: Literal["user", "assistant"]
    item_id: str                    # Server‑assigned identifier
    text: str | None = None         # Filled once transcript is ready

@dataclass
class ConversationState:
    """All mutable data the session needs — nothing more, nothing less."""
    history: List[Turn] = field(default_factory=list)         # Ordered log
    waiting: dict[str, asyncio.Future] = field(default_factory=dict)  # Pending transcript fetches
    summary_count: int = 0

    latest_tokens: int = 0          # Window size after last reply
    summarising: bool = False       # Guard so we don’t run two summaries at once

トランスクリプトを確認するためのクイックヘルパー：

In [4]:
def print_history(state) -> None:
    """Pretty-print the running transcript so far."""
    print("—— Conversation so far ———————————————")
    for turn in state.history:
        text_preview = (turn.text or "").strip().replace("\n", " ")
        print(f"[{turn.role:<9}] {text_preview}  ({turn.item_id})")
    print("——————————————————————————————————————————")

### 3.2 · ストリーミング音声
生のPCM-16マイクデータを直接Realtime APIにストリーミングします。

パイプラインは: mic ─► async.Queue ─► WebSocket ─► Realtime API

#### 3.2.1 マイク入力のキャプチャ
以下の処理を行うコルーチンから始めます：

* デフォルトのマイクを**24 kHz、モノラル、PCM‑16**で開く（Realtimeが受け入れる[フォーマット](https://platform.openai.com/docs/api-reference/realtime-sessions/create#realtime-sessions-create-input_audio_format)の一つ）。  
* ストリームを**約40ms**のブロックに分割する。  
* 各ブロックを`asyncio.Queue`に投入し、別のタスク（次のセクション）がOpenAIに転送できるようにする。

In [5]:
async def mic_to_queue(pcm_queue: asyncio.Queue[bytes]) -> None:
    """
    Capture raw PCM‑16 microphone audio and push ~CHUNK_DURATION_MS chunks
    to *pcm_queue* until the surrounding task is cancelled.

    Parameters
    ----------
    pcm_queue : asyncio.Queue[bytes]
        Destination queue for PCM‑16 frames (little‑endian int16).
    """
    blocksize = int(SAMPLE_RATE_HZ * CHUNK_DURATION_MS / 1000)

    def _callback(indata, _frames, _time, status):
        if status:                               # XRuns, device changes, etc.
            print("⚠️", status, file=sys.stderr)
        try:
            pcm_queue.put_nowait(bytes(indata))  # 1‑shot enqueue
        except asyncio.QueueFull:
            # Drop frame if upstream (WebSocket) can’t keep up.
            pass

    # RawInputStream is synchronous; wrap in context manager to auto‑close.
    with sd.RawInputStream(
        samplerate=SAMPLE_RATE_HZ,
        blocksize=blocksize,
        dtype="int16",
        channels=1,
        callback=_callback,
    ):
        try:
            # Keep coroutine alive until cancelled by caller.
            await asyncio.Event().wait()
        finally:
            print("⏹️  Mic stream closed.")

#### 3.2.2 オーディオチャンクをAPIに送信する

マイクタスクは現在、生のPCM-16ブロックで`asyncio.Queue`を満たしています。  
次のステップ：そのキューからチャンクを取り出し、**base-64エンコード**を行い（プロトコルはJSON安全なテキストを要求します）、各ブロックを`input_audio_buffer.append`イベントとしてRealtime WebSocketに送信します。

In [6]:
# Helper function to encode audio chunks in base64
b64 = lambda blob: base64.b64encode(blob).decode()

async def queue_to_websocket(pcm_queue: asyncio.Queue[bytes], ws):
    """Read audio chunks from queue and send as JSON events."""
    try:
        while (chunk := await pcm_queue.get()) is not None:
            await ws.send(json.dumps({
                "type": "input_audio_buffer.append",
                "audio": b64(chunk),
            }))
    except websockets.ConnectionClosed:
        print("WebSocket closed – stopping uploader")

#### 3.2.3 受信イベントの処理
音声がサーバーに到達すると、Realtime APIは**同じ**WebSocket上でJSONイベントのストリームをプッシュバックします。  
これらのイベントを理解することは以下の点で重要です：

* ライブ転写の表示  
* ユーザーへの増分音声の再生  
* 正確な[`Conversation State`](https://platform.openai.com/docs/api-reference/realtime-server-events/conversation/created)の維持（後でコンテキストトリミングが機能するため）  


| イベントタイプ | 到着タイミング | 重要性 | 典型的なハンドラーロジック |
|------------|-----------------|---------------|-----------------------|
| **`session.created`** | WebSocketハンドシェイク直後 | セッションが開いていることを確認し、`session.id`を提供する。 | トレーサビリティのためにIDをログに記録し、接続を確認する。 |
| **`session.updated`** | `session.update`呼び出しを送信した後 | サーバーが新しいセッション設定を適用したことを確認する。 | エコーされた設定を検査し、ローカルキャッシュを更新する。 |
| **`conversation.item.created`** (user) | ユーザーが話し終えてから数ms後（クライアントVADが発火） | タイムラインスロットを予約する；転写はまだ**`null`**の可能性がある。 | `state.history`に「転写待ち」とマークされた*プレースホルダー*ユーザーターンを挿入する。 |
| **`conversation.item.retrieved`** | 約100-300ms後、音声転写が完了した時点 | 最終的なユーザー転写（タイミング付き）を提供する。 | プレースホルダーを転写で置き換え、必要に応じて表示する。 |
| **`response.audio.delta`** | アシスタントが話している間、20-60ms毎 | PCM-16音声チャンク（およびオプションの増分テキスト）をストリーミングする。 | 各チャンクをバッファリングして再生する；オプションでコンソールに部分テキストを表示する。 |
| **`response.done`** | アシスタントの最後のトークンの後 | 音声とテキストの両方が完了したことを通知；使用統計を含む。 | アシスタントターンを確定し、`state.latest_tokens`を更新し、使用量をログに記録する。 |
| **`conversation.item.deleted`** | `conversation.item.delete`でプルーニングする際 | ターンが削除され、サーバー上のトークンが解放されたことを確認する。 | コンテキストウィンドウがサーバーと一致するように、ローカルでも削除をミラーリングする。 |

### 3.3 要約のタイミングを検出する
Realtimeモデルは**大きな32kトークンのウィンドウ**を保持しますが、モデルにより多くのコンテキストを詰め込むと、その制限に達するずっと前に品質が低下する可能性があります。

我々の目標：実行中のウィンドウが安全な閾値（ノートブックのデフォルトは**2,000トークン**）に近づいたら**自動要約**を行い、その後、置き換えられたターンをローカル*および*サーバー側の両方で削除することです。

`response.done`で返される`latest_tokens`を監視します。これが`SUMMARY_TRIGGER`を超え、`KEEP_LAST_TURNS`より多くのターンがある場合、バックグラウンドで要約コルーチンを開始します。

最後の2ターンを除くすべてを単一のフランス語段落に圧縮し、その後：

1. その段落を会話の先頭に新しいアシスタントメッセージとして挿入します。

2. 要約に使用されたメッセージアイテムを削除します。

後で音声エージェントに要約の言語を尋ねて、Realtime API会話コンテキストへの要約挿入が成功したかをテストします。

In [7]:
async def run_summary_llm(text: str) -> str:
    """Call a lightweight model to summarise `text`."""
    resp = await asyncio.to_thread(lambda: openai.chat.completions.create(
        model=SUMMARY_MODEL,
        temperature=0,
        messages=[
            {"role": "system", "content": "Summarise in French the following conversation "
                            "in one concise paragraph so it can be used as "
                            "context for future dialogue."},
            {"role": "user", "content": text},
        ],
    ))
    return resp.choices[0].message.content.strip()

重要な実装の詳細：
- 要約はASSISTANTメッセージではなく、SYSTEMメッセージとして追加されます。テストの結果、長時間の会話中に要約にASSISTANTメッセージを使用すると、モデルが音声応答からテキスト応答に誤って切り替わってしまう可能性があることが判明しました。要約にSYSTEMメッセージを使用することで（追加のカスタム指示も含めることができます）、これらがコンテキスト設定の指示であることをモデルに明確に伝え、進行中のユーザー・アシスタント間のやり取りのモダリティを誤って採用することを防ぎます。

In [8]:
async def summarise_and_prune(ws, state):
    """Summarise old turns, delete them server‑side, and prepend a single summary
    turn locally + remotely."""
    state.summarising = True
    print(
        f"⚠️  Token window ≈{state.latest_tokens} ≥ {SUMMARY_TRIGGER}. Summarising…",
    )
    old_turns, recent_turns = state.history[:-KEEP_LAST_TURNS], state.history[-KEEP_LAST_TURNS:]
    convo_text = "\n".join(f"{t.role}: {t.text}" for t in old_turns if t.text)
    
    if not convo_text:
        print("Nothing to summarise (transcripts still pending).")
        state.summarising = False

    summary_text = await run_summary_llm(convo_text) if convo_text else ""
    state.summary_count += 1
    summary_id = f"sum_{state.summary_count:03d}"
    state.history[:] = [Turn("assistant", summary_id, summary_text)] + recent_turns
    
    print_history(state)    

    # Create summary on server
    await ws.send(json.dumps({
        "type": "conversation.item.create",
        "previous_item_id": "root",
        "item": {
            "id": summary_id,
            "type": "message",
            "role": "system",
            "content": [{"type": "input_text", "text": summary_text}],
        },
    }))

    # Delete old items
    for turn in old_turns:
        await ws.send(json.dumps({
            "type": "conversation.item.delete",
            "item_id": turn.item_id,
        }))

    print(f"✅ Summary inserted ({summary_id})")
    
    state.summarising = False

以下の関数は、時間をかけて転写結果をポーリングすることを可能にします。これは、ユーザーの音声がすぐに転写されない場合に有用で、後で最終結果を取得することができます。

In [9]:
async def fetch_full_item(
    ws, item_id: str, state: ConversationState, attempts: int = 1
):
    """
    Ask the server for a full conversation item; retry up to 5× if the
    transcript field is still null.  Resolve the waiting future when done.
    """
    # If there is already a pending fetch, just await it
    if item_id in state.waiting:
        return await state.waiting[item_id]

    fut = asyncio.get_running_loop().create_future()
    state.waiting[item_id] = fut

    await ws.send(json.dumps({
        "type": "conversation.item.retrieve",
        "item_id": item_id,
    }))
    item = await fut

    # If transcript still missing retry (max 5×)
    if attempts < 5 and not item.get("content", [{}])[0].get("transcript"):
        await asyncio.sleep(0.4 * attempts)
        return await fetch_full_item(ws, item_id, state, attempts + 1)

    # Done – remove the marker
    state.waiting.pop(item_id, None)
    return item


## 4. エンドツーエンドワークフローのデモンストレーション

以下の2つのセルを実行してインタラクティブセッションを開始してください。セルを中断すると記録が停止されます。

> **注意:**  
> このノートブックでは、要約のデモンストレーションを素早く行うために `SUMMARY_TRIGGER = 2000` と `KEEP_LAST_TURNS = 2` を使用しています。  
> 本番環境では、アプリケーションのニーズに基づいてこれらの値を調整する必要があります。  
> - 一般的な `SUMMARY_TRIGGER` は **20,000～32,000トークン** の範囲で設定され、使用ケースにおいてより大きなコンテキストでパフォーマンスがどの程度低下するかによって決まります。

In [10]:
# Audio/config knobs
SAMPLE_RATE_HZ    = 24_000   # Required by pcm16
CHUNK_DURATION_MS = 40       # chunk size for audio capture
BYTES_PER_SAMPLE  = 2        # pcm16 = 2 bytes/sample
SUMMARY_TRIGGER   = 2_000    # Summarise when context ≥ this
KEEP_LAST_TURNS   = 2       # Keep these turns verbatim
SUMMARY_MODEL     = "gpt-4o-mini"  # Cheaper, fast summariser

In [11]:
# --------------------------------------------------------------------------- #
# Realtime session                                                          #
# --------------------------------------------------------------------------- #
async def realtime_session(model="gpt-realtime", voice="shimmer", enable_playback=True):
    """
    Main coroutine: connects to the Realtime endpoint, spawns helper tasks,
    and processes incoming events in a big async‑for loop.
    """
    state = ConversationState()  # Reset state for each run

    pcm_queue: asyncio.Queue[bytes] = asyncio.Queue()
    assistant_audio: List[bytes] = []

    # ----------------------------------------------------------------------- #
    # Open the WebSocket connection to the Realtime API                       #
    # ----------------------------------------------------------------------- #
    url = f"wss://api.openai.com/v1/realtime?model={model}"
    headers = {"Authorization": f"Bearer {openai.api_key}"}

    async with websockets.connect(url, extra_headers=headers, max_size=1 << 24) as ws:
        # ------------------------------------------------------------------- #
        # Wait until server sends session.created                             #
        # ------------------------------------------------------------------- #
        while json.loads(await ws.recv())["type"] != "session.created":
            pass
        print("session.created ✅")

        # ------------------------------------------------------------------- #
        # Configure session: voice, modalities, audio formats, transcription  #
        # ------------------------------------------------------------------- #
        await ws.send(json.dumps({
            "type": "session.update",
            "session": {
                "type": "realtime",
                model: "gpt-realtime",
                "voice": voice,
                "modalities": ["audio", "text"],
                "input_audio_format": "pcm16",
                "output_audio_format": "pcm16",
                "input_audio_transcription": {"model": "gpt-4o-transcribe"},
            },
        }))

        # ------------------------------------------------------------------- #
        # Launch background tasks: mic capture → queue → websocket            #
        # ------------------------------------------------------------------- #
        mic_task = asyncio.create_task(mic_to_queue(pcm_queue))
        upl_task = asyncio.create_task(queue_to_websocket(pcm_queue, ws))

        print("🎙️ Speak now (Ctrl‑C to quit)…")

        try:
            # ------------------------------------------------------------------- #
            # Main event loop: process incoming events from the websocket         #
            # ------------------------------------------------------------------- #
            async for event_raw in ws:
                event = json.loads(event_raw)
                etype = event["type"]

                # --------------------------------------------------------------- #
                # User just spoke ⇢ conversation.item.created (role = user)        #
                # --------------------------------------------------------------- #
                if etype == "conversation.item.created" and event["item"]["role"] == "user":
                    item = event["item"]
                    text = None
                    if item["content"]:
                        text = item["content"][0].get("transcript")
                    
                    state.history.append(Turn("user", event["item"]["id"], text))
                    
                    # If transcript not yet available, fetch it later
                    if text is None:
                        asyncio.create_task(fetch_full_item(ws, item["id"], state))

                # --------------------------------------------------------------- #
                # Transcript fetched ⇢ conversation.item.retrieved                 #
                # --------------------------------------------------------------- #
                elif etype == "conversation.item.retrieved":
                    content = event["item"]["content"][0]
                    # Fill missing transcript in history
                    for t in state.history:
                        if t.item_id == event["item"]["id"]:
                            t.text = content.get("transcript")
                            break

                # --------------------------------------------------------------- #
                # Assistant audio arrives in deltas                               #
                # --------------------------------------------------------------- #
                elif etype == "response.audio.delta":
                    assistant_audio.append(base64.b64decode(event["delta"]))

                # --------------------------------------------------------------- #
                # Assistant reply finished ⇢ response.done                        #
                # --------------------------------------------------------------- #
                elif etype == "response.done":
                    for item in event["response"]["output"]:
                        if item["role"] == "assistant":
                            txt = item["content"][0]["transcript"]
                            state.history.append(Turn("assistant", item["id"], txt))
                            # print(f"\n🤖 {txt}\n")
                    state.latest_tokens = event["response"]["usage"]["total_tokens"]
                    print(f"—— response.done  (window ≈{state.latest_tokens} tokens) ——")
                    print_history(state)
                    
                    # Fetch any still‑missing user transcripts
                    for turn in state.history:
                        if (turn.role == "user"
                            and turn.text is None
                            and turn.item_id not in state.waiting):
                            asyncio.create_task(
                                fetch_full_item(ws, turn.item_id, state)
                            )

                    # Playback collected audio once reply completes
                    if enable_playback and assistant_audio:
                        simpleaudio.play_buffer(b"".join(assistant_audio), 1, BYTES_PER_SAMPLE, SAMPLE_RATE_HZ)
                        assistant_audio.clear()

                    # Summarise if context too large – fire in background so we don't block dialogue
                    if state.latest_tokens >= SUMMARY_TRIGGER and len(state.history) > KEEP_LAST_TURNS and not state.summarising:
                        asyncio.create_task(summarise_and_prune(ws, state))

        except KeyboardInterrupt:
            print("\nStopping…")
        finally:
            mic_task.cancel()
            await pcm_queue.put(None)
            await upl_task

In [ ]:
# Run the realtime session (this cell blocks until you stop it)
await realtime_session()

```raw
session.created ✅
🎙️ 今話してください（Ctrl‑Cで終了）…
—— response.done  (window ≈979 tokens) ——
—— これまでの会話 ———————————————
[user     ] 簡単な話をしてもらえますか？  (item_BTuMOcpUqp8qknKhLzlkA)
[assistant] 昔々、居心地の良い小さな村に、いつもトラブルに巻き込まれるWhiskersという名前の猫がいました。ある晴れた日、Whiskersは庭で神秘的に光る石を見つけました。好奇心旺盛な彼は前足でそれを触ると、ポン！その石は彼に鳥と話す能力を与えました。Whiskersと新しい鳥の友達たちは壮大な冒険をし、謎を解き、村を探検しました。そしてその日から、Whiskersは村で最も冒険好きな猫として知られるようになりました。おしまい。  (item_BTuMPRWxqpv0ph6QM46DK)
——————————————————————————————————————————
—— response.done  (window ≈2755 tokens) ——
—— これまでの会話 ———————————————
[user     ] 簡単な話をしてもらえますか？  (item_BTuMOcpUqp8qknKhLzlkA)
[assistant] 昔々、居心地の良い小さな村に、いつもトラブルに巻き込まれるWhiskersという名前の猫がいました。ある晴れた日、Whiskersは庭で神秘的に光る石を見つけました。好奇心旺盛な彼は前足でそれを触ると、ポン！その石は彼に鳥と話す能力を与えました。Whiskersと新しい鳥の友達たちは壮大な冒険をし、謎を解き、村を探検しました。そしてその日から、Whiskersは村で最も冒険好きな猫として知られるようになりました。おしまい。  (item_BTuMPRWxqpv0ph6QM46DK)
[user     ] とても面白い話を3つ教えてもらえますか？  (item_BTuNN64LdULM21OyC4vzN)
[assistant] もちろん、笑えるお話に飛び込んでみましょう：  **話その1：** Bennyという忘れっぽいパン屋がいて、大きな結婚式のために100個のケーキを焼きました。しかし当日、彼はそれらをどこに置いたか忘れてしまいました！町全体が行方不明のケーキを探すのに参加し、Bennyが自分のパントリーだと思って隣人のガレージに保管していたことが判明しました。結婚式は町全体のケーキパーティーになりました！  **話その2：** Sparkyといういたずら好きな犬がいて、いたずらをするのが大好きでした。ある日、彼は飼い主の電話をきしむおもちゃと交換し、吠え声、きしみ音、混乱した通話の陽気な混乱を引き起こしました。Sparkyの飼い主は結局、郵便配達員と全てきしみ音で完全な会話をすることになりました！  **話その3：** 小さな町で、Pollyというオウムが早口言葉を暗唱することで地元の有名人になりました。ある日、Pollyは市長に早口言葉決闘を挑みました。舌がもつれて笑っている市長は、Pollyを公式の町の道化師に任命しました。Pollyは誇らしげに鳴き、町は何日も笑い声で響きました。  (item_BTuNNpNxki5ynSQ5c3Xsa)
——————————————————————————————————————————
⚠️  Token window ≈2755 ≥ 2000. 要約中…
—— これまでの会話 ———————————————
[assistant] L'utilisateur a demandé une histoire rapide, et l'assistant a raconté celle d'un chat nommé Whiskers qui, après avoir trouvé une pierre mystérieuse dans son jardin, a obtenu le pouvoir de parler aux oiseaux. Avec ses nouveaux amis oiseaux, Whiskers a vécu de grandes aventures, résolvant des mystères et explorant le village, devenant ainsi le chat le plus aventurier du village.  (sum_001)
[user     ] とても面白い話を3つ教えてもらえますか？  (item_BTuNN64LdULM21OyC4vzN)
[assistant] もちろん、笑えるお話に飛び込んでみましょう：  **話その1：** Bennyという忘れっぽいパン屋がいて、大きな結婚式のために100個のケーキを焼きました。しかし当日、彼はそれらをどこに置いたか忘れてしまいました！町全体が行方不明のケーキを探すのに参加し、Bennyが自分のパントリーだと思って隣人のガレージに保管していたことが判明しました。結婚式は町全体のケーキパーティーになりました！  **話その2：** Sparkyといういたずら好きな犬がいて、いたずらをするのが大好きでした。ある日、彼は飼い主の電話をきしむおもちゃと交換し、吠え声、きしみ音、混乱した通話の陽気な混乱を引き起こしました。Sparkyの飼い主は結局、郵便配達員と全てきしみ音で完全な会話をすることになりました！  **話その3：** 小さな町で、Pollyというオウムが早口言葉を暗唱することで地元の有名人になりました。ある日、Pollyは市長に早口言葉決闘を挑みました。舌がもつれて笑っている市長は、Pollyを公式の町の道化師に任命しました。Pollyは誇らしげに鳴き、町は何日も笑い声で響きました。  (item_BTuNNpNxki5ynSQ5c3Xsa)
——————————————————————————————————————————
✅ 要約が挿入されました (sum_001)
—— response.done  (window ≈2147 tokens) ——
—— これまでの会話 ———————————————
[assistant] L'utilisateur a demandé une histoire rapide, et l'assistant a raconté celle d'un chat nommé Whiskers qui, après avoir trouvé une pierre mystérieuse dans son jardin, a obtenu le pouvoir de parler aux oiseaux. Avec ses nouveaux amis oiseaux, Whiskers a vécu de grandes aventures, résolvant des mystères et explorant le village, devenant ainsi le chat le plus aventurier du village.  (sum_001)
[user     ] とても面白い話を3つ教えてもらえますか？  (item_BTuNN64LdULM21OyC4vzN)
[assistant] もちろん、笑えるお話に飛び込んでみましょう：  **話その1：** Bennyという忘れっぽいパン屋がいて、大きな結婚式のために100個のケーキを焼きました。しかし当日、彼はそれらをどこに置いたか忘れてしまいました！町全体が行方不明のケーキを探すのに参加し、Bennyが自分のパントリーだと思って隣人のガレージに保管していたことが判明しました。結婚式は町全体のケーキパーティーになりました！  **話その2：** Sparkyといういたずら好きな犬がいて、いたずらをするのが大好きでした。ある日、彼は飼い主の電話をきしむおもちゃと交換し、吠え声、きしみ音、混乱した通話の陽気な混乱を引き起こしました。Sparkyの飼い主は結局、郵便配達員と全てきしみ音で完全な会話をすることになりました！  **話その3：** 小さな町で、Pollyというオウムが早口言葉を暗唱することで地元の有名人になりました。ある日、Pollyは市長に早口言葉決闘を挑みました。舌がもつれて笑っている市長は、Pollyを公式の町の道化師に任命しました。Pollyは誇らしげに鳴き、町は何日も笑い声で響きました。  (item_BTuNNpNxki5ynSQ5c3Xsa)
[user     ]   (item_BTuPLaCv8ATdIwAQ2rLgO)
[assistant] もちろんです！私たちの間で最初に提供した要約はフランス語でした。  (item_BTuPLa7BaSQToGCVOmfBK)
```

---
私たちはVoice AIと会話をしました。数回のやり取りの後、総トークン数がSUMMARY_MAXに達し、会話要約ステップがトリガーされました。これにより、以前のメッセージの要約が生成されました。

総メッセージ数がN = 4だったため、最初のN - 2 = 2個のメッセージを要約しました：
```txt
—— Conversation so far ———————————————
[user     ] Can you tell me a quick story?  (item_BTuMOcpUqp8qknKhLzlkA)
[assistant] Once upon a time, in a cozy little village, there was a cat named Whiskers who was always getting into trouble. One sunny day, Whiskers found a mysterious glowing stone in the garden. Curious, he pawed at it, and poof! The stone granted him the ability to talk to birds. Whiskers and his new bird friends had grand adventures, solving mysteries and exploring the village. And from that day on, Whiskers was known as the most adventurous cat in the village. The end.  (item_BTuMPRWxqpv0ph6QM46DK)
```

その後、フランス語で要約を作成し、`root: true`フラグを使用して会話履歴に挿入しました。これにより、要約が会話の最初のメッセージとして表示されることが保証されました。その後、`"type": "conversation.item.delete"`を使用して、要約された元のアイテムを削除しました。

要約の挿入を検証するため、Voice AIに要約がどの言語で書かれているかを尋ねました。正しく応答しました：

```txt
[assistant] Sure! The first summary I provided between us was in French.  (item_BTuPLa7BaSQToGCVOmfBK)
```

## 5 · 実世界での応用

コンテキスト要約は**長時間実行される音声体験**に有用です。  
以下にユースケースのアイデアを示します：

| ユースケース | 付加価値 | 有用性 |
|----------|-------------|------------|
| **カスタマーサポート音声ボット** | 24時間365日の自然な電話応答システム；チケット要約の自動生成 | 長時間の顧客通話を要約して効率的な引き継ぎと記録管理を実現し、エージェントの作業負荷を軽減して応答品質を向上させます。 |
| **語学チューター** | 修正フィードバック付きのリアルタイム会話練習 | 学習者の進歩を追跡し、繰り返される間違いをハイライトすることで、個人に合わせたフィードバックとより効果的な言語習得を可能にします。 |
| **AIセラピスト / コーチ** | セッションを記憶する安全で常に利用可能な聞き手 | 重要なトピックと感情的なトーンを記憶してセッション間の継続性を維持し、より共感的で効果的な体験をサポートします。 |
| **会議アシスタント** | ライブ転写 + Slackでの簡潔なアクションアイテム要約 | 長時間の会議を実行可能な要約に凝縮し、チームメンバーの時間を節約して重要なポイントを見逃さないようにします。 |

## 6 · 次のステップと参考資料
ノートブックを試して、コンテキスト要約をあなたのアプリケーションに統合してみてください。

試すことができるいくつかの項目：
| 試してみること… | 学べること |
|-----------|------------------|
| **要約のA/Bテスト**<br/>要約を*オン*と*オフ*にして評価スイートを実行する。 | トリミングが実際にあなたのドメインで品質を向上させるかどうか、そしてレイテンシとコストにどう影響するか。 |
| **要約スタイルの変更**<br/>システムプロンプトを箇条書き、JSON、英語対フランス語などに変更する。 | 下流のアシスタントがどの形式を最もよく吸収するか、言語選択が後続の回答にどう影響するか。 |
| **閾値の調整**<br/>`SUMMARY_TRIGGER_TOKENS`（2k → 8k）で調整する。 | モデルドリフトと要約オーバーヘッドの間の最適なバランス。 |
| **コストトレーシング**<br/>要約前後の`usage.total_tokens`をログに記録する。 | 具体的なROI：会話1時間あたりのトークン節約量。 |


### リソース：
- [OpenAI Realtime Guide](https://platform.openai.com/docs/guides/realtime)
- [OpenAI Realtime Conversations](https://platform.openai.com/docs/guides/realtime-conversations)
- [OpenAI Realtime API Reference](https://platform.openai.com/docs/api-reference/realtime)
- [Voice AI and Voice Agents](https://voiceaiandvoiceagents.com/)